In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'

import json
import pandas as pd
pd.options.display.max_colwidth = 200
from pandas.io.json import json_normalize
import numpy as np
import os
from os import walk
import re
import feather
from collections import Counter

In [3]:
## Change directory to where all of the json files are located so we can recursively open files
## within this folder and read the json objects
os.chdir('/Users/zacklarsen/Dropbox (Inference Analytics)/Inference Analytics Team Folder/Zack Work/boldchat/')

path = '/Users/zacklarsen/Dropbox (Inference Analytics)/Inference Analytics Team Folder/Zack Work/boldchat/'

f = []; # Initialize an empty list if we want to record all filenames
examples = []

for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames) # This adds the file names to a list
    for file in filenames:
        if file.lower().endswith((".json")):
            with open(file, 'r') as f:
                data = json.load(f)
                try:
                    example = data['Data']
                    #print(example)
                    examples.extend(example)
                except:
                    pass    
            
        else:
            print('File ending is not .json:','\''+file+'\'')
    break
    
    
#len(examples) 
JSONdf = json_normalize(examples)
JSONdf['Created'] = pd.to_datetime(JSONdf['Created'])
JSONdf.sort_values(['Created'],inplace=True)

# Remove html tags
JSONdf['Text'] = JSONdf['Text'].str.replace('<[^>]+>', '')

# Remove non-English conversations. This reduces data from 295257 to 218158 rows
JSONdf = JSONdf[JSONdf['LanguageCode'] == 'en-US']

JSONdf.shape #(295257, 20)



# Store as .csv to different folder
os.chdir('/Users/zacklarsen/Dropbox (Inference Analytics)/Inference Analytics Team Folder/Zack Work/')
JSONdf.to_csv('MasterJSON.csv')

# Store efficiently as feather format
feather.write_dataframe(JSONdf, 'MasterFeather.feather')

(218158, 20)

In [4]:
JSONdf.head(3)

,AccountID,CannedMessageID,ChatID,ChatMessageID,Created,Deleted,HasAutoTranslationError,Hidden,LanguageCode,MessageKey,Name,OriginalMessageID,PersonID,PersonType,RowNumber,SearchName,SearchText,SystemMessage,Text,VisitorFacing
10021,450758035848538255,None,447517168823042369,106504829878320930,2016-04-01 00:00:52.840,None,None,None,en-US,None,ahmed shaaban,None,447508220678983806,1,2851,None,None,None,ok,None
10018,450758035848538255,None,447517168823042369,209167656383451667,2016-04-01 00:02:47.487,None,None,None,en-US,None,ahmed shaaban,None,447508220678983806,1,2848,None,None,None,FLR1012ST,None
10017,450758035848538255,None,447517168823042369,70991537481037942,2016-04-01 00:03:12.107,None,None,None,en-US,None,ahmed shaaban,None,447508220678983806,1,2847,None,None,None,what should be the delivery time,None


In [5]:
JSONdf.to_csv('JSONdf.csv')

In [72]:
#os.path.getsize('MasterJSON.csv') #44254930
#os.path.getsize('MasterFeather.feather') #52576264


## Compare timing
# %timeit csvJSONdf = pd.read_csv('MasterJSON.csv')
## 854 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Read feather format file
# %timeit featherJSONdf = feather.read_dataframe('MasterFeather.feather')
## 189 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [2]:
JSONdf = feather.read_dataframe('MasterFeather.feather')

In [3]:
JSONdf['ChatID'].value_counts().head()

447617973405364153    116
447610868356936474    114
447758726101386059    109
447558064076927556    106
447540901806344057    106
Name: ChatID, dtype: int64

In [7]:
JSONdf['Text'].head()

10021                                                 ok
10018                                          FLR1012ST
10017                   what should be the delivery time
10007    Hello, my name is Michael, how may I help you? 
10006                         I find Thermocouple Type T
Name: Text, dtype: object

In [8]:
## Extract part numbers from text

pattern = re.compile(r"([A-Z]+[A-Z-]{2,})")
parts = []
partsColumn = []

for chatline in JSONdf['Text']:
    match = re.findall(pattern, chatline)
    if len(match) > 0:
        parts.append(match)
        partsColumn.append(match)
    else:
        partsColumn.append('None')

len(parts)
len(partsColumn)

36158

218158

In [9]:
## Create value counts of nested list of parts
sorted_count = Counter(sublist[0] for sublist in parts).most_common(100)

## Extract just the parts name for the top 100 most common parts, excluding its count
top100 = [[i][0][0] for i in sorted_count]

In [10]:
if 'OMEGA' in top100: top100.remove('OMEGA')
if 'THANKS' in top100: top100.remove('THANKS')

In [11]:
top100

['RTD',
 'RMA',
 'USB',
 'FMA',
 'PR-',
 'NPT',
 'VDC',
 'DAQ',
 'DPG',
 'FTB',
 'UPS',
 'HTML',
 'PXM',
 'FL-',
 'SSR',
 'PID',
 'DIN',
 'NIST',
 'DASG',
 'PLC',
 'USA',
 'TXDIN',
 'FDT-',
 'DAS',
 'VAC',
 'FMG',
 'OM-',
 'FTB-',
 'DPF',
 'AWG',
 'FLR',
 'ICIN',
 'RDXL',
 'SERIES',
 'PHE-',
 'PSI',
 'PFA',
 'RTD-',
 'CAD',
 'FMA-',
 'OM-EL-USB-',
 'OMB-DAQ-',
 'TC-',
 'WIRE',
 'CNI',
 'PDF',
 'LCM',
 'SSRL',
 'FPD',
 'CASS-',
 'XCIB',
 'HSTC',
 'CNPT',
 'VAT',
 'SMPW',
 'PT-',
 'HHF',
 'OB-',
 'ASAP',
 'B-S',
 'ISO',
 'PSW-',
 'GPM',
 'CHAL-',
 'OM-CP-IFC',
 'OM-SQ',
 'CAL-',
 'LSC',
 'PVC',
 'OM-DAQ-USB-',
 'UWTC-REC',
 'LVDT',
 'SLE',
 'FDT',
 'TT-K-',
 'LPM',
 'SRFG-',
 'FMA-A',
 'FP-',
 'PROBES',
 'HDC',
 'ICSS-',
 'HHP',
 'UWTC',
 'LVCN',
 'USD',
 'TEC',
 'ITHX-SD',
 'TH-',
 'YES',
 'TEMPERATUREG',
 'ACC',
 'DMD',
 'DPI',
 'SRT',
 'CAXL-',
 'ROHS',
 'INET-']

In [22]:
JSONdf = pd.read_csv('JSONdf.csv')
#JSONdf.columns
JSONdf.drop('Unnamed: 0',axis=1,inplace=True)

/Users/zacklarsen/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
#JSONdf['Text'].head(30)

# JSONdf['ChatID'].value_counts().index[0] ## This is the longest chat
Chat = JSONdf[JSONdf['ChatID'] == JSONdf['ChatID'].value_counts().index[0]]
Chat.sort_values(['Created','PersonID'],inplace=True)
Chat

/Users/zacklarsen/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,AccountID,CannedMessageID,ChatID,ChatMessageID,Created,Deleted,HasAutoTranslationError,Hidden,LanguageCode,MessageKey,Name,OriginalMessageID,PersonID,PersonType,RowNumber,SearchName,SearchText,SystemMessage,Text,VisitorFacing
94914,450758035848538255,3.937517e+18,447617973405364153,3567565307256355365,2016-07-26 16:01:09.753,NaN,NaN,NaN,en-US,NaN,Steve,NaN,893246988109179758,0,1588.0,NaN,NaN,NaN,"Hello, my name is Steve, how may I help you?",NaN
94915,450758035848538255,NaN,447617973405364153,144092808802041875,2016-07-26 16:01:34.810,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1587.0,NaN,NaN,NaN,Hi Steve.,NaN
94917,450758035848538255,NaN,447617973405364153,2636736593755820,2016-07-26 16:02:02.563,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1585.0,NaN,NaN,NaN,I've a project that involves using a lot of long thermocouples,NaN
94918,450758035848538255,NaN,447617973405364153,375625008628316378,2016-07-26 16:02:14.997,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1584.0,NaN,NaN,NaN,Long = about 70 to 80 ft thermocouples.,NaN
94921,450758035848538255,NaN,447617973405364153,349454320571412152,2016-07-26 16:03:02.547,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1581.0,NaN,NaN,NaN,"When I tried comparing the accuracy of longer thermocouple with smaller length, there is always a difference in reading between two thermocouples",NaN
94922,450758035848538255,NaN,447617973405364153,745995328358092827,2016-07-26 16:03:09.867,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1580.0,NaN,NaN,NaN,although the tips are at same place,NaN
94923,450758035848538255,NaN,447617973405364153,781889258200224438,2016-07-26 16:03:22.887,NaN,NaN,NaN,en-US,NaN,Steve,NaN,893246988109179758,0,1579.0,NaN,NaN,NaN,"hello Sachin , what is the application ? max and min temp ?",NaN
94926,450758035848538255,NaN,447617973405364153,155372503342494777,2016-07-26 16:04:36.783,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1576.0,NaN,NaN,NaN,It is embedded in concrete pavement -- so it would be few degrees different than normal outside temperature,NaN
94928,450758035848538255,NaN,447617973405364153,797993706244509639,2016-07-26 16:04:48.067,NaN,NaN,NaN,en-US,NaN,Sachin,NaN,447513642824405647,1,1573.0,NaN,NaN,NaN,-15 to 50 degree celsius,NaN
94937,450758035848538255,NaN,447617973405364153,990337535866917663,2016-07-26 16:06:38.437,NaN,NaN,NaN,en-US,NaN,Steve,NaN,893246988109179758,0,1557.0,NaN,NaN,NaN,"we can offer a T type ready made thermocouple that is 80 ft long ,",NaN


In [24]:
for entry in Chat['Text']:
    print(entry)

Hello, my name is Steve, how may I help you?
Hi Steve.
I've a project that involves using a lot of long thermocouples
Long = about 70 to 80 ft thermocouples.
When I tried comparing the accuracy of longer thermocouple with smaller length, there is always a difference in reading between two thermocouples
although the tips are at same place
hello Sachin , what is the application ? max and min temp ?
It is embedded in concrete pavement -- so it would be few degrees different than normal outside temperature
-15 to 50 degree celsius
we can offer a T type ready made thermocouple that is 80 ft long , 
We have Type N
So my question is, i there a way to account for these drifts?
Actually if you have an email id, it would be easier to show you the graphs.
are you welding&nbsp; the N type wire &nbsp; ?&nbsp; how much of a difference are you getting? is this an Omega part ?
Yeah.
We are using bare fine wire thermocouple
my email is strapani@omega.com 
what is the AWG&nbsp;wire size ?
24
I will send

In [90]:
Chats = JSONdf.loc[:,['ChatID','Text']]
Chats.sort_values('ChatID',inplace=True)

In [91]:
Chats.tail()

,ChatID,Text
218132,447842608295552754,Or I suppose I could tape it down to a pcb/substrate and put it through the oven.
218123,447842608295552754,Hello. I'm looking for a thermocouple to profile my reflow ovens. Peak temperatures of 400Â°C with total runtime of about 5 minutes. Do you have any suggestions?
218130,447842608295552754,Flexible would be preferable I think. I'll need to hook the probe to the conveyor belt.
218145,447842608295552754,All set ?
218143,447842608295552754,Type K is best choice


In [45]:
## Subset the data based on type

#Chats.select_dtypes(include=[int])

#Chats.select_dtypes(include=[object])

# for item in Chats.Text:
#     if type(item) == str:
#         pass
#         #print(str(item))
#     else:
#         print('Not a string')
#         print(item)
    #print(str(item))

In [110]:
# Remove Text entries that are null
Chats = Chats[~Chats.Text.isnull()]

In [112]:
## Group the chats together by ChatID and join the individual messages together to
## form one large string
                     
Chats = pd.DataFrame(Chats.groupby('ChatID')['Text'].apply(','.join))

# Alternatively:
#Chats = pd.DataFrame(Chats.groupby('ChatID')['Text'].apply(lambda x: ','.join(x)))

In [114]:
Chats = JSONdf.loc[:,['Created','ChatID','Text']]
Chats.sort_values(['ChatID','Created'],ascending=[0,1],inplace=True)
Chats = Chats[~Chats.Text.isnull()]

Chats = pd.DataFrame(Chats.groupby('ChatID')['Text'].apply(','.join))

Chats.to_csv('Chats.csv')

In [115]:
# os.chdir('')
Chats = pd.read_csv('Chats.csv')

In [117]:
Chats.head()

,ChatID,Text
0,447517168823042369,"ok,FLR1012ST,what should be the delivery time"
1,447517292449367530,"Hello, my name is Michael, how may I help you? ,I find Thermocouple Type T,Size 1.5 SQMM = 30 mm.,I live in Rayong (Thailand),yes, do you have the part number? ,Kindly review as there are a lot o..."
2,447517294027314935,"Ok thanks,Is there anything else I can help you with?,What digital instruments are you looking for?,All sales direct from Singapore to india shipment,I want your supplier in India,I want digital v..."
3,447517378126795012,"is this sensor come with digital display,You are welcome, Mr Ravi,ok fine thank you!,we will send you the catalog as well,yes,ya i have registerd,hii,will you please send me the detailes of same,y..."
4,447517413202164194,"we ship from our singapore site to Indiia/Australia/South East Asia.,super,perfect,can you tell me how to place an order and payment terms,Hi Mr Subash, you just need to online ordering and procee..."


## Topic modeling with latent dirichlet allocation (LDA)

In [118]:
## Create a list of strings, where each string is a conversation from the chat logs
doc_list = Chats['Text']

In [120]:
import gensim
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
import codecs

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [121]:
tokenizer = RegexpTokenizer(r'\w+')

# create english stop words list
en_stop = set(stopwords.words('english'))
en_stop.add('omega')
en_stop.add('thank')
en_stop.add('you')
en_stop.add('?&nbsp')
print(en_stop)


# create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

{'few', 'by', 'its', 'below', 'was', 'mightn', 're', 'haven', 'for', 'any', 'over', 'same', 'above', 'after', 'you', 'whom', 'are', 'were', 'there', 'very', 'had', 'all', 'now', 'yourself', 'before', 'doesn', 'weren', 'further', 'is', 'can', 'do', 'to', 'in', 'her', 'ma', 'been', 'yourselves', 'some', 'your', 'no', 'if', 'more', 'own', 'does', 'should', 'and', 'at', 'who', 'they', 'just', 'about', 'having', 'hers', 'that', 'between', 'mustn', 'has', 'here', 'them', 'out', 'am', 'their', 'an', 'my', 'o', 'd', 'll', 'because', 'other', 'me', 'doing', 'how', 'hadn', 'won', 'itself', 'each', 'so', 'which', 'yours', 'once', 'again', 'ours', 'being', 'shan', 'nor', 'myself', 'such', 'hasn', 'don', 't', 'thank', 'what', 'him', 'a', 'than', 'theirs', 'y', 'isn', 'why', 'these', 'didn', 'then', 'he', 'too', 'it', 'the', 'where', 'did', 'aren', 'wasn', 'from', 'needn', 'only', 'm', 'into', 'will', 'but', 'of', 'when', 'omega', 'those', 'up', 'off', 'most', 'on', 's', 'through', 'during', 'agains

In [122]:
# list for tokenized documents in loop
texts = []

# loop through document list -- make sure to specify doc_set1 or doc_set2
for i in doc_list:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        
        # remove stop words form tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        
        # add tokens to list
        texts.append(stemmed_tokens)

In [123]:
# turn our tokenized documents into a id <> term dictionary
dictionary = corpora.Dictionary(texts)

In [124]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [125]:
## A higher alpha (closer to 1) will yield many topics for a given document. 
## A lower value of alpha (closer to 0) will yield only only topic per document.
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=30,
                                           id2word = dictionary,
                                           passes = 1,
                                           alpha = 0.1)

In [126]:
topics = ldamodel.print_topics(num_topics=30,num_words=15)

for topic in topics:
    print('Topic number',topic[0],':')
    print(topic[1])

Topic number 0 :
0.041*"strain" + 0.021*"gaug" + 0.013*"bridg" + 0.012*"ã" + 0.010*"termin" + 0.009*"need" + 0.008*"wind" + 0.007*"wire" + 0.006*"ich" + 0.006*"e" + 0.006*"ì" + 0.006*"hh804" + 0.005*"simon" + 0.005*"assist" + 0.005*"nbsp"
Topic number 1 :
0.038*"34" + 0.030*"1" + 0.020*"2" + 0.017*"4" + 0.014*"need" + 0.014*"probe" + 0.013*"3" + 0.011*"com" + 0.010*"look" + 0.010*"fit" + 0.010*"6" + 0.010*"would" + 0.009*"diamet" + 0.009*"length" + 0.009*"http"
Topic number 2 :
0.019*"pdf" + 0.015*"look" + 0.015*"help" + 0.013*"use" + 0.012*"http" + 0.012*"com" + 0.012*"hello" + 0.012*"www" + 0.011*"may" + 0.009*"name" + 0.009*"need" + 0.008*"gaug" + 0.008*"ok" + 0.007*"hi" + 0.007*"thank"
Topic number 3 :
0.036*"pressur" + 0.027*"0" + 0.022*"output" + 0.015*"rang" + 0.014*"would" + 0.011*"use" + 0.011*"help" + 0.011*"transduc" + 0.010*"signal" + 0.010*"need" + 0.010*"measur" + 0.010*"look" + 0.009*"hello" + 0.009*"sensor" + 0.009*"may"
Topic number 4 :
0.020*"panel" + 0.017*"would" + 

## Be careful- printing the lines below will take a long time

In [127]:
## Beta values associated with each topic, which tell us how probable it is that
## the given topic is associated with a given document.

## This data structure below is the 'gamma', which tells us the beta values associated with each document
for i in range(len(ldamodel[corpus])):
    print('Document:',i,ldamodel[corpus][i])

Document: 0 [(0, 0.014286683007587031), (1, 0.014287225330272348), (2, 0.014286778451652426), (3, 0.014286709619199704), (4, 0.01428661252361758), (5, 0.014289591138835298), (6, 0.01428670521645741), (7, 0.014287021307631697), (8, 0.014286427090980566), (9, 0.014288022265801944), (10, 0.01428673179478411), (11, 0.014287969073025004), (12, 0.014287338121375738), (13, 0.014286507170582732), (14, 0.014286731232118875), (15, 0.014286633589893297), (16, 0.014290222868973388), (17, 0.014303384052900287), (18, 0.01429070942206041), (19, 0.014287947077848265), (20, 0.014290656200869844), (21, 0.014287982441603933), (22, 0.01428680321841363), (23, 0.014288239745682291), (24, 0.014286208343521309), (25, 0.014286755767996153), (26, 0.014288567163930498), (27, 0.01428759208083708), (28, 0.014288955697175979), (29, 0.58564228898437121)]
Document: 1 [(1, 0.12669121784517975), (8, 0.07835288267865477), (11, 0.39845445463522383), (13, 0.0541185915271322), (14, 0.014305432405168142), (23, 0.20238923081

Document: 72 [(6, 0.016945344986541343), (7, 0.093314598513951574), (10, 0.21402023058135933), (12, 0.39215851154771053), (13, 0.077157698315102358), (23, 0.18417548946604329)]
Document: 73 [(17, 0.066214986483273605), (19, 0.89002717283198884)]
Document: 74 [(3, 0.67101890446226753), (9, 0.012920915718812568), (10, 0.052853185438609958), (19, 0.030988065863040707), (25, 0.13350806922160297), (29, 0.085692039327122882)]
Document: 75 [(1, 0.8073800928214987), (5, 0.044650256896999869), (18, 0.089345739035900887), (25, 0.010627657453501067), (29, 0.037355809816728432)]
Document: 76 [(0, 0.041706590130633735), (1, 0.06404458490537461), (2, 0.052344155414520144), (3, 0.20160521197462042), (7, 0.12068623452911229), (8, 0.012646881709703619), (10, 0.042894015407768861), (12, 0.1109975720381335), (14, 0.073660036244774935), (19, 0.13021535198030076), (25, 0.13808578116718895)]
Document: 77 [(1, 0.17742556461856021), (3, 0.10615462068142252), (11, 0.1396557163630977), (21, 0.1383974689488541),

Document: 150 [(7, 0.33125995988517992), (11, 0.53462457014907006), (26, 0.04972577818691034)]
Document: 151 [(3, 0.23572246568624006), (11, 0.036344743204445008), (12, 0.051512201131806545), (13, 0.053245449052881262), (17, 0.062722271756756098), (26, 0.48236083729852397), (27, 0.053356245781868032)]
Document: 152 [(0, 0.012500138480651724), (1, 0.012500961518340648), (2, 0.012500638313093835), (3, 0.012500456717681852), (4, 0.012500579296708765), (5, 0.012501358462232369), (6, 0.012501143932529594), (7, 0.63747477515104056), (8, 0.01250043424040825), (9, 0.012500474422108278), (10, 0.012500976407235402), (11, 0.012501421586111099), (12, 0.012500679833602939), (13, 0.012500642724471643), (14, 0.012500887875717353), (15, 0.012500565502260903), (16, 0.012500597022274872), (17, 0.012502934242260079), (18, 0.012500865875466472), (19, 0.012500965832470243), (20, 0.012500798984021899), (21, 0.012501084227763878), (22, 0.012500464951444103), (23, 0.012500505685642309), (24, 0.012500191394093

Document: 216 [(10, 0.1254191297297145), (11, 0.32665521540690989), (13, 0.047824025601269485), (16, 0.16932880331727476), (17, 0.070140340856458372), (20, 0.011453617766326097), (26, 0.21273137133335676), (27, 0.025334250901141851)]
Document: 217 [(10, 0.16232854523656454), (11, 0.65242528266865374), (22, 0.012773668845478363), (29, 0.11829859761152124)]
Document: 218 [(7, 0.21300259765189775), (20, 0.17765024076862218), (21, 0.1521603480335027), (27, 0.37592229613696265)]
Document: 219 [(3, 0.27204357861637041), (6, 0.12446033444949725), (10, 0.10915636308639627), (11, 0.17248122371529054), (12, 0.29804401847095946)]
Document: 220 [(4, 0.10705061667272203), (8, 0.035871897586970902), (9, 0.095629356944596705), (13, 0.50050093502627968), (16, 0.16031266172549966), (20, 0.063031149523477825)]
Document: 221 [(5, 0.068536894233599002), (6, 0.013899733250912129), (7, 0.11103846575952993), (13, 0.20889777842376381), (17, 0.21861938956644911), (23, 0.21162702859753985), (27, 0.1517311507389

Document: 312 [(1, 0.13145163547397726), (3, 0.5976739425797748), (4, 0.015908508198165947), (10, 0.033409104304616813), (11, 0.01750632519133298), (12, 0.054447363131252209), (19, 0.043453664421649482), (25, 0.086824808620602101)]
Document: 313 [(3, 0.62927680972986944), (13, 0.20255615476728075), (17, 0.11305204488622335)]
Document: 314 [(7, 0.25419231298379602), (12, 0.59842218181793017)]
Document: 315 [(7, 0.43276734140983142), (11, 0.21395885960885441), (14, 0.18399440830817587), (16, 0.10880210337230328)]
Document: 316 [(0, 0.012500138295609683), (1, 0.012500482412901847), (2, 0.012500945796402626), (3, 0.012500713235845199), (4, 0.012501325269834612), (5, 0.012500994781588906), (6, 0.01250096037654665), (7, 0.4542119050534672), (8, 0.012500545892945065), (9, 0.012501437320239782), (10, 0.01250178571063691), (11, 0.012501397117064696), (12, 0.012500894511864674), (13, 0.012500980814227186), (14, 0.012501147235477714), (15, 0.012500801671548085), (16, 0.012501967513940107), (17, 0

...

Document: 16115 [(3, 0.22907554389013252), (4, 0.051987342765566547), (17, 0.19656270984468058), (18, 0.28197117319555698), (22, 0.16111830823378545), (24, 0.047698557569973536)]
Document: 16116 [(1, 0.25346317556041309), (2, 0.15758678383966127), (13, 0.086947119000781864), (14, 0.037529035914866019), (16, 0.23041329312080255), (17, 0.070721700263943948), (22, 0.10582700021187796)]
Document: 16117 [(3, 0.14764930644258029), (11, 0.064155268331583234), (18, 0.46023655004145014), (19, 0.21601422068535506), (25, 0.046142227610400904)]
Document: 16118 [(17, 0.89999397597911046)]
Document: 16119 [(1, 0.7363292561914776)]
Document: 16120 [(1, 0.031954304434611135), (3, 0.16485393500419709), (7, 0.083786035358737035), (8, 0.36238114399231303), (9, 0.2720885387875307), (12, 0.036880028823952769), (15, 0.01216503592718398), (28, 0.014131473169702379)]
Document: 16121 [(3, 0.22314021683454624), (5, 0.07759517772320032), (7, 0.18633125848889262), (17, 0.20456725133093134), (23, 0.242564000356949

Document: 16374 [(5, 0.3750324339645012), (10, 0.19478355094302655), (14, 0.061706643614145601), (28, 0.16709699814807896), (29, 0.14817794727948105)]
Document: 16375 [(4, 0.014679836496776153), (9, 0.26136585169981486), (12, 0.049325530658549263), (14, 0.027953203124244839), (15, 0.34216660379401337), (26, 0.24780344660170125), (29, 0.035600171875924576)]
Document: 16376 [(18, 0.80092053235451588), (19, 0.052701498545785909), (25, 0.10418278673480381)]
Document: 16377 [(20, 0.83471940077424556), (29, 0.048606405021349434)]
Document: 16378 [(0, 0.020212271700349799), (6, 0.025048830030381455), (14, 0.046681144849580622), (18, 0.1120720260665488), (22, 0.047318893031709296), (24, 0.020867579666787622), (26, 0.49825334482334122), (27, 0.034720471397134495), (29, 0.15922646066595009)]
Document: 16379 [(1, 0.12031569058413269), (16, 0.14214557449009463), (25, 0.10371098171993141), (27, 0.32620876106809499), (29, 0.11529222898740846)]
Document: 16380 [(1, 0.12883586507959446), (4, 0.0847368

Document: 16621 [(7, 0.026228009872152295), (8, 0.052983663797342588), (11, 0.054526386251131959), (13, 0.020933077330710814), (20, 0.7519720531295544), (23, 0.022975211605897775), (29, 0.040497831903442326)]
Document: 16622 [(3, 0.10113786658945757), (13, 0.083928104929743882), (23, 0.13098096455000244), (26, 0.27695320450383387), (28, 0.16001052375233435), (29, 0.13787492049543462)]
Document: 16623 [(1, 0.089944161707108825), (2, 0.22246221193284516), (8, 0.12707583346846893), (16, 0.054119796078176227), (21, 0.1103601075919752), (27, 0.35602886717164872)]
Document: 16624 [(0, 0.01834167702112495), (13, 0.08227620285948864), (16, 0.28273084230668649), (17, 0.28738029110538232), (23, 0.1923471009764591), (27, 0.073123250510920307), (28, 0.029464753281583198)]
Document: 16625 [(5, 0.060687407082078232), (13, 0.6615415445377596), (21, 0.089082593534282911), (27, 0.11063594071197154), (28, 0.031982593111595181), (29, 0.021322574019118506)]
Document: 16626 [(2, 0.076977315697624768), (6, 

Document: 16898 [(17, 0.14073751461296871), (18, 0.58986645740867738), (19, 0.14684359111071194), (20, 0.076932223526247179)]
Document: 16899 [(1, 0.028239227650649006), (2, 0.030820851971796986), (5, 0.012469105265039349), (8, 0.050805536544909179), (9, 0.046667655785468304), (13, 0.13331633457177741), (22, 0.010129149726524855), (27, 0.67022493605651801)]
Document: 16900 [(3, 0.54367070980782406), (10, 0.065949812009684874), (12, 0.049495666577996318), (13, 0.18622405514954651), (17, 0.10540679597109277), (18, 0.02500585902654481)]
Document: 16901 [(18, 0.20691621649818409), (19, 0.70557133172453812)]
Document: 16902 [(17, 0.23785648854368993), (19, 0.21206105425870711), (20, 0.088911242263482682), (26, 0.25989177433603533), (27, 0.084791705218977623), (29, 0.067498778397875894)]
Document: 16903 [(1, 0.20496193238993471), (4, 0.056784837905262738), (6, 0.014983484693299076), (11, 0.31532677703162143), (13, 0.040152600212205826), (17, 0.21310895963664547), (19, 0.092410941073100814), 

Document: 17138 [(2, 0.034021599871403664), (3, 0.16077341601120879), (7, 0.2351879566028007), (8, 0.038031451899998708), (9, 0.1674865034527305), (15, 0.26175161483960269), (19, 0.037710982417424405), (23, 0.025744008662999862)]
Document: 17139 [(10, 0.3782915352231887), (14, 0.088352111872914252), (15, 0.045631206821464182), (20, 0.20978733694723789), (25, 0.14240361418125586), (28, 0.084459081070278857)]
Document: 17140 [(7, 0.0828764414388206), (8, 0.029803375633003153), (20, 0.64450445810292134), (23, 0.16756796184665723), (29, 0.030595983532623848)]
Document: 17141 [(5, 0.020980463387206952), (12, 0.56105199472847578), (13, 0.34182290443456587), (17, 0.019095757361617376), (20, 0.022321164379809036)]
Document: 17142 [(7, 0.64106870682266803), (18, 0.12556861788242804)]
Document: 17143 [(1, 0.12000355753911696), (10, 0.11128114002473503), (16, 0.09959448284967802), (17, 0.047155596020596546), (18, 0.13602417640093711), (23, 0.1154535591921272), (25, 0.03657802496292914), (26, 0.03